In [48]:
import pandas as pd
import numpy as np
#eplus_out_file = "/home/zhiangz/Documents/HVAC-RL-Control/src/eplus-env/eplus_env/envs/eplus_models/iw_v96/learning/idf/tmy3Weather_base_res/eplusout.csv";
eplus_out_file = '/home/zhiangz/Documents/HVAC-RL-Control/src/iw_ctrl_runs_v2/2/Eplus-env-IW-tmy3Weather-v9601-res1/Eplus-env-sub_run37/output/eplusout.csv'
occupant_count_colname = 'EMS:IW Occupy Mode Flag [](TimeStep)'
ppd_colname = 'EMS:IW Average PPD [%](TimeStep)'
energy_colname = 'EMS:IW Calculated Heating Demand [KW](TimeStep)'
stptbylogic_colname = 'EMS:IW Effective IAT Setpoint by Logics [C](TimeStep)'
iat_colname = 'EMS:IW North Zone Average Temperature [C](TimeStep)'
timestep = 5 # Minutes

In [49]:
def getFullColname(colnames, tgtname):
    for colname in colnames:
        if tgtname in colname:
            return colname;

In [50]:
df = pd.read_csv(eplus_out_file)
df_colnames = list(df.columns.values)
occupant_count_colname = getFullColname(df_colnames, occupant_count_colname)
ppd_colname = getFullColname(df_colnames, ppd_colname)
energy_colname = getFullColname(df_colnames, energy_colname)
stptbylogic_colname = getFullColname(df_colnames, stptbylogic_colname)
iat_colname = getFullColname(df_colnames, iat_colname);

In [51]:
occp_ppd = df[ppd_colname].loc[df[occupant_count_colname] > 0]
valid_energy = df[energy_colname].loc[np.isnan(df[energy_colname]) != 1]
iat_stpt_diff = df[iat_colname].loc[df[occupant_count_colname] == 0] - df[stptbylogic_colname].loc[df[occupant_count_colname] == 0]

In [52]:
iat_stpt_diff

0       -2.312008
1       -2.338468
2       -2.383978
3       -2.436839
4       -2.488531
5       -2.533802
6       -2.570120
7       -2.596711
8       -2.616218
9       -2.629706
10      -2.598141
11      -2.519742
12      -2.394882
13      -2.220874
14      -2.084827
15      -1.963451
16      -1.881034
17      -1.817199
18      -1.767823
19      -1.730408
20      -1.701673
21      -1.678966
22      -1.669070
23      -1.662676
24      -1.655791
25      -1.648240
26      -1.632388
27      -1.624631
28      -1.620029
29      -1.614992
           ...   
25889    4.785935
25890    4.770325
25891    4.754449
25892    4.738624
25893    4.722924
25894    4.714541
25895    4.710813
25896    4.707904
25897    4.693960
25898    4.636293
25899    4.579637
25900    4.536428
25901    4.500122
25902    4.467867
25903    4.445929
25904    4.430561
25905    4.417440
25906    4.402545
25907    4.383807
25908    4.361902
25909    4.338064
25910    4.313386
25911    4.288610
25912    4.264211
25913    4

In [53]:
mean_occp_ppd = np.mean(occp_ppd)
std_occp_ppd = np.std(occp_ppd)
total_energy = np.sum(valid_energy) * (timestep/60.0)
mean_iat_notmet = np.mean(iat_stpt_diff)
std_iat_notmet = np.std(iat_stpt_diff);

In [54]:
print ('Total HVAC energy consumption is %d kHW, mean occupied PPD is %0.02f%%, std occupied PPD is %0.02f%%'\
       ', mean IAT not met during unoccupied hour is %0.02f, std IAT not met during unoccupied hour is %0.02f.'
      %(total_energy, mean_occp_ppd, std_occp_ppd, mean_iat_notmet, std_iat_notmet))

Total HVAC energy consumption is 50707 kHW, mean occupied PPD is 7.65%, std occupied PPD is 3.96%, mean IAT not met during unoccupied hour is 1.90, std IAT not met during unoccupied hour is 1.64.


In [47]:
np.histogram(occp_ppd)

(array([6015, 4694, 1865,  530,  168,  129,   68,   13,    6,    3]),
 array([  5.27060861,   9.28433289,  13.29805717,  17.31178146,
         21.32550574,  25.33923002,  29.3529543 ,  33.36667859,
         37.38040287,  41.39412715,  45.40785143]))